In [1]:
import gym
import numpy as np
import torch

from environments import *

from stable_baselines3 import SAC, HerReplayBuffer
from stable_baselines3.common.cmd_util import make_vec_env

c:\Users\ssuro\anaconda3\lib\site-packages\torch\utils\tensorboard\__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
c:\Users\ssuro\anaconda3\lib\site-packages\tensorboard\compat\proto\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
c:\Users\ssuro\anaconda3\lib\site-packages\tensorboard\compat\proto\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
c:\Users\ssuro\anaconda3\lib\site-packages\tensorboard\compat\proto\tensor_shape_pb2

In [2]:
env = make_vec_env("G1Goal-v1", 1)

c:\Users\ssuro\anaconda3\lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\ssuro\anaconda3\lib\site-packages\gym\wrappers\step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [3]:
model = SAC(
    "MultiInputPolicy", 
    env, 
    verbose=1,
    replay_buffer_class=HerReplayBuffer,
    replay_buffer_kwargs=dict(
      n_sampled_goal=4,
      goal_selection_strategy="future",
      max_episode_length=1000,
      online_sampling=True,
    ),
    policy_kwargs=dict(
        net_arch=[512, 512, 512, 512], 
        activation_fn=torch.nn.ELU,
        n_critics=5,
    ),
    learning_rate=1e-4,
    learning_starts=1000,
    tensorboard_log="./logs/sac+her/"
)

Using cuda device


In [9]:
model.set_parameters("trained_models/g1-forward/sac+her", env=env)

In [6]:
model.learn(
    total_timesteps=1e5, 
    log_interval=100,
)

c:\Users\ssuro\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\ssuro\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
c:\Users\ssuro\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(


Logging to ./logs/sac+her/SAC_1
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 141       |
|    ep_rew_mean     | -1.45e+03 |
| time/              |           |
|    episodes        | 100       |
|    fps             | 21        |
|    time_elapsed    | 653       |
|    total_timesteps | 14110     |
| train/             |           |
|    actor_loss      | -3.27     |
|    critic_loss     | 490       |
|    ent_coef        | 0.275     |
|    ent_coef_loss   | -20.7     |
|    learning_rate   | 0.0001    |
|    n_updates       | 13109     |
----------------------------------


KeyboardInterrupt: 

In [8]:
model.save("trained_models/g1-forward/sac+her")